In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
import os

In [2]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

feature_view = fs.get_feature_view(
    name='tmdb_fv',
    version=3,
)

2025-01-08 13:35:20,905 INFO: Initializing external client
2025-01-08 13:35:20,906 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 13:35:22,115 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [3]:
mr= project.get_model_registry()

retrieved_model=mr.get_model(
    name="movie_rating_model_2",
    version=1,
)

saved_model_dir=retrieved_model.download()

In [4]:
retrieved_xgboost_model=XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir+"/model.json")

In [5]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

movie_fg = fs.get_feature_group(
    name='tmdb_upcoming_movies',
    version=3,
)

df_new = movie_fg.read()

df_new

2025-01-08 13:35:44,938 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 13:35:44,944 INFO: Initializing external client
2025-01-08 13:35:44,944 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 13:35:46,181 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.88s) 


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year
0,1217624,The Evolution of Horror Cinema Worldwide,0.0,0.0,Post Production,2025-09-18 00:00:00+00:00,1000.0,90.0,1000.0,tt15679984,en,The Evolution of Horror Cinema Worldwide,Why do people no longer see this type of stori...,4.633,"""The whole Evolution of Global Terror cinema i...","Documentary, Horror","David Conill, Jhon Jatenjor's Interviews","Colombia, Spain","English, Español","Kaiti Wallen, Zoe Berriatúa, Rosmary Yaneva, D...",Jhon Jatenjor,None,Jhon Jatenjor,"David Conill, Ray Barlett",None,NaN,NaN,/q4GDeVerLjRNzlHfAFPaRoE28PF.jpg,2025
1,1266805,The Musicians,0.0,0.0,Post Production,2025-03-26 00:00:00+00:00,0.0,102.0,0.0,None,fr,Les musiciens,Astrid Thompson finally succeeds in fulfilling...,0.817,None,Drama,"Les Films Velvet, Baxter Films",France,Français,"Valérie Donzelli, Frédéric Pierrot, Mathieu Sp...",Grégory Magne,None,Grégory Magne,"Pierre-Louis Garnon, Frédéric Jouve",Grégoire Hetzel,NaN,NaN,None,2025
2,1315297,Dire Pines,0.0,0.0,Post Production,2025-01-24 00:00:00+00:00,0.0,0.0,3000.0,None,en,Dire Pines,While grieving the loss of his younger brother...,3.056,None,"Horror, Drama, Fantasy",Monochromatic Pillow Pictures,Canada,English,"Isaac Stacey, Matthew Trueman","Jim Couroux, Taylor Allison",Camille Poon,Jim Couroux,"Jim Couroux, Taylor Allison, David Daniels Won...",Jim Couroux,NaN,NaN,/mt2YmSyY6qz0VHrKsfATbd5YCPr.jpg,2025
3,931942,BLKNWS: Terms and Conditions,0.0,0.0,Post Production,2025-01-24 00:00:00+00:00,0.0,113.0,0.0,tt17497960,en,BLKNWS: Terms and Conditions,Preeminent West African curator and scholar Fu...,0.001,None,None,"Participant, A24",United States of America,English,"Shaunette Renée Wilson, Peter Jay Fernandez, H...",Kahlil Joseph,Bradford Young,"Saidiya Hartman, Kahlil Joseph, Irvin Hunt","Kahlil Joseph, Onye Anyanwu, Nicolas Gonda, An...",None,NaN,NaN,/7wQgClzJTgzdKAqyNDfhzyt0NjR.jpg,2025
4,1405751,Coroner to the Stars,0.0,0.0,Post Production,2025-02-20 00:00:00+00:00,0.0,83.0,0.0,None,en,Coroner to the Stars,Coroner to the Stars chronicles the extraordin...,1.058,None,Documentary,"Liaison Pictures, Strangely Compelling Multime...",United States of America,"English, Português","Dr. Thomas Noguchi, Paul Guyet, Dan Medina, Dr...","Keita Ideno, Ben Hethcoat","Michelle Lawler, Erica Chan",None,"Chiaki Yanagimoto, John Henry Hinkel, Tommy An...","Mike Schanzlin, Mike Griffin",NaN,NaN,/9doStrDXyrhRpY2o78AugRcgJGP.jpg,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1389619,Parkside,0.0,0.0,Post Production,2025-01-15 00:00:00+00:00,0.0,0.0,150.0,tt32760279,en,Parkside,"After a night out, Olivia (Gillian Garvie) cal...",1.400,None,Drama,None,None,English,"Mattaya Braun, Benjamin Jorgensen, Gillian Garvie",Clay Cesmistruk,None,Clay Cesmistruk,Clay Cesmistruk,None,NaN,NaN,None,2025
756,1235619,Les Bodin’s partent en vrille,0.0,0.0,Post Production,2025-03-19 00:00:00+00:00,0.0,0.0,0.0,tt32192665,fr,Les Bodin’s partent en vrille,None,1.142,None,Comedy,"Cheyenne Productions, M6 Films",France,Français,"Vincent Dubois, Jean-Christophe Fraiscinet, Lé...",Frédéric Forestier,"Stéphane Le Parc, Christian Abomnes","Vincent Dubois, Jean-Christophe Fraiscinet","Claude Cyndecki, Frantz Richard",None,NaN,NaN,/4DUeoyYHwLIgzeCw8eszbjd0KgG.jpg,2025
757,1379877,Azaad,0.0,0.0,Post Production,2025-01-17 00:00:00+00:00,0.0,0.0,0.0,None,hi,आज़ाद,A skilled horse rider and his family are fight...,3.769,None,"Drama, Action","RSVP Movies, Guy in the Sky Pictures",India,हिन्दी,"Jiya Imran Amin, Amaan Devgn, Akshay Anand Koh...",Abhishek Kapoor,None,"K. Hariharan, Abhishek Kapoor, Ritesh Shah, Su...","Pragya Kapoor, Ronnie 

In [6]:
# Have budget
df_b = df_new[(df_new['budget'] > 10000000.0)]
df_b["imdb_votes"]=400

df_b

2025-01-08 13:35:53,375 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year
57,1100988,28 Years Later,0.0,0.0,Post Production,2025-06-16 00:00:00+00:00,0.0,0.0,75000000.0,tt10548174,en,28 Years Later,It's been almost three decades since the rage ...,79.863,In 28 days it began. In 28 weeks it spread. In...,"Horror, Science Fiction","DNA Films, Decibel Films, Columbia Pictures, BFI","United Kingdom, United States of America",English,"Ralph Fiennes, Joe Blakemore, Nathan Hall, Edv...",Danny Boyle,Anthony Dod Mantle,"Danny Boyle, Alex Garland","Danny Boyle, Peter Rice, Cillian Murphy, Berna...",None,NaN,400,/hVKHzr4GwSw0FepqhqQ0DDiYHNY.jpg,2025
60,1101383,Flowervale Street,0.0,0.0,Post Production,2026-03-12 00:00:00+00:00,0.0,0.0,85000000.0,tt27165187,en,Flowervale Street,A family in the '80s start to notice bizarre h...,4.194,None,"Adventure, Thriller","Jackson Pictures, Bad Robot, Warner Bros. Pict...",United States of America,English,"Jordan Alexa Davis, Anne Hathaway, Christian C...",David Robert Mitchell,Mike Gioulakis,David Robert Mitchell,"David Robert Mitchell, Chris Bender, J.J. Abra...",None,NaN,400,None,2026
100,1294480,Генг Бенг 3,0.0,0.0,Post Production,2024-05-27 00:00:00+00:00,0.0,0.0,290000000.0,None,ru,Генг Бенг 3,None,0.000,None,"Action, Comedy, Adventure",GangBang Studios Co.,None,None,"Гомункул, Cillian Murphy, Ryan Gosling, Ryan R...",Хлазу,None,Хлазу,"Christopher Nolan, Хлазу",None,NaN,400,None,2024
108,931285,Mortal Kombat 2,0.0,0.0,Post Production,2025-10-23 00:00:00+00:00,0.0,0.0,68000000.0,tt17490712,en,Mortal Kombat 2,A sequel to the 2021 film 'Mortal Kombat'.,16.116,None,"Action, Fantasy, Adventure","New Line Cinema, Atomic Monster, Broken Road P...",United States of America,English,"Desmond Chiam, Tati Gabrielle, Josh Lawson, Jo...",Simon McQuoid,Stephen F. Windon,Jeremy Slater,"Michael Clear, Simon McQuoid, Toby Emmerich, J...",Benjamin Wallfisch,NaN,400,/yN1bQ8XLh7Q2bEu3YBtp68vUeXl.jpg,2025
198,1277016,The Queen of Heaven,0.0,0.0,Post Production,2025-04-04 00:00:00+00:00,0.0,196.0,22000000.0,tt25049058,en,Η βασίλισσα των ουρανών,The life of the Virgin Mary from the Crucifixi...,2.189,None,Drama,"Y.S. Pictures, Crisis Cinema, Enorama, BO Film...",Greece,English,"George Angelopoulos, Maria Aliferi, Yorgo Cons...",Yiannis Stravolaimos,None,Yiannis Stravolaimos,"Ali Bakkioui, Yiannis Stravolaimos",None,NaN,400,/zY2vIC5q9waeSKDrYjAQnP6CyTz.jpg,2025
203,1292465,Совет Андреев,0.0,0.0,Post Production,2024-08-09 00:00:00+00:00,0.0,0.0,190000000.0,None,ru,Совет Андреев,None,1.400,None,"Thriller, Comedy, Drama, Science Fiction, Fantasy","JPD4, GangBang Studios Co.",Russia,None,"Ezra Miller, Ryan Reynolds, Cillian Murphy, To...","JPD4, Дариус",None,Дариус,"Christopher Nolan, Дариус",None,NaN,400,None,2024
252,1238159,Tarot,0.0,0.0,Post Production,2024-05-01 00:00:00+00:00,0.0,92.0,12500000.0,None,en,Tarot,When a group of friends recklessly violates th...,17.302,None,Horror,"Alloy Entertainment, Capstone Pictures, Ground...",United States of America,English,"Olwen Fouéré, Humberly Gonzalez, Staša Nikolić...","Anna Halberg, Spenser Cohen",None,"Anna Halberg, Spenser Cohen","Leslie Morgenstein, Elysa Koplovitz Dutton, Sc...",None,NaN,400,None,2024
257,1127674,La città proibita,0.0,0.0,Post Production,2025-03-13 00:00:00+00:00,0.0,0.0,18218260.0,tt27766538,it,La città proibita,Set in the cosmopolitan melting pot of Rome’s ...,1.991,None,Action,"Wildside, Goon Films, PiperFilm",Italy,Italiano,"Luca Zingaretti, Yaxi Liu, Sabrina Ferilli, Ma...",Gabriele Mainetti,Paolo Carnera,"Stefano Bises, Gabriele Mainetti, Davide Serino","Lorenzo Gangarossa, Gabriele Mainetti, Mario G...",None,NaN,400,None,2025
308,936075,Michael,0.0,0.0,Post Production,2025-10-01 00:00:00+00:00,0

In [7]:
from utils import new_features

df1= new_features(df_b)

In [8]:
df1.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year,first_producer,first_actor,first_company
57,1100988,28 Years Later,0.0,0.0,Post Production,2025-06-16 00:00:00+00:00,0.0,0.0,75000000.0,tt10548174,0,28 Years Later,It's been almost three decades since the rage ...,79.863,In 28 days it began. In 28 weeks it spread. In...,"Horror, Science Fiction","DNA Films, Decibel Films, Columbia Pictures, BFI","United Kingdom, United States of America",English,"Ralph Fiennes, Joe Blakemore, Nathan Hall, Edv...",Danny Boyle,Anthony Dod Mantle,"Danny Boyle, Alex Garland","Danny Boyle, Peter Rice, Cillian Murphy, Berna...",None,NaN,400,/hVKHzr4GwSw0FepqhqQ0DDiYHNY.jpg,2025,4,18,5
60,1101383,Flowervale Street,0.0,0.0,Post Production,2026-03-12 00:00:00+00:00,0.0,0.0,85000000.0,tt27165187,0,Flowervale Street,A family in the '80s start to notice bizarre h...,4.194,None,"Adventure, Thriller","Jackson Pictures, Bad Robot, Warner Bros. Pict...",United States of America,English,"Jordan Alexa Davis, Anne Hathaway, Christian C...",David Robert Mitchell,Mike Gioulakis,David Robert Mitchell,"David Robert Mitchell, Chris Bender, J.J. Abra...",None,NaN,400,None,2026,7,10,11
100,1294480,Генг Бенг 3,0.0,0.0,Post Production,2024-05-27 00:00:00+00:00,0.0,0.0,290000000.0,None,4,Генг Бенг 3,None,0.000,None,"Action, Comedy, Adventure",GangBang Studios Co.,None,None,"Гомункул, Cillian Murphy, Ryan Gosling, Ryan R...",Хлазу,None,Хлазу,"Christopher Nolan, Хлазу",None,NaN,400,None,2024,3,22,8
108,931285,Mortal Kombat 2,0.0,0.0,Post Production,2025-10-23 00:00:00+00:00,0.0,0.0,68000000.0,tt17490712,0,Mortal Kombat 2,A sequel to the 2021 film 'Mortal Kombat'.,16.116,None,"Action, Fantasy, Adventure","New Line Cinema, Atomic Monster, Broken Road P...",United States of America,English,"Desmond Chiam, Tati Gabrielle, Josh Lawson, Jo...",Simon McQuoid,Stephen F. Windon,Jeremy Slater,"Michael Clear, Simon McQuoid, Toby Emmerich, J...",Benjamin Wallfisch,NaN,400,/yN1bQ8XLh7Q2bEu3YBtp68vUeXl.jpg,2025,18,4,14
198,1277016,The Queen of Heaven,0.0,0.0,Post Production,2025-04-04 00:00:00+00:00,0.0,196.0,22000000.0,tt25049058,0,Η βασίλισσα των ουρανών,The life of the Virgin Mary from the Crucifixi...,2.189,None,Drama,"Y.S. Pictures, Crisis Cinema, Enorama, BO Film...",Greece,English,"George Angelopoulos, Maria Aliferi, Yorgo Cons...",Yiannis Stravolaimos,None,Yiannis Stravolaimos,"Ali Bakkioui, Yiannis Stravolaimos",None,NaN,400,/zY2vIC5q9waeSKDrYjAQnP6CyTz.jpg,2025,0,7,20


In [13]:
df_encoded = df1.copy()
df_encoded = df_encoded[["runtime", "release_year", "imdb_votes", "budget", "genres", "first_producer", "first_actor", "first_company", "original_language"]]
df_encoded['genres'] = df_encoded['genres'].str.split(', ')  # Split by comma and space
genres_encoded = df_encoded['genres'].explode()  # Flatten the list
one_hot = pd.get_dummies(genres_encoded).groupby(level=0).sum()

one_hot.columns = one_hot.columns.str.lower()  # Lowercase column names
one_hot.columns = one_hot.columns.str.replace(' ', '_')  # Replace spaces with underscores

# Add missing columns if not already in the DataFrame
all_genres=['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction', 'tv_movie', 'thriller', 'war', 'western']
for col in all_genres:
        if col not in one_hot.columns:
            one_hot[col] = 0  # or df_encoded[col] = False if they were boolean flags

# Merge one-hot encoding with the original DataFrame
train_features = pd.concat([df_encoded, one_hot], axis=1)
train_features=train_features.drop(["genres"], axis=1)
train_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 57 to 745
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   runtime            23 non-null     float64
 1   release_year       23 non-null     int32  
 2   imdb_votes         23 non-null     int64  
 3   budget             23 non-null     float64
 4   first_producer     23 non-null     int64  
 5   first_actor        23 non-null     int64  
 6   first_company      23 non-null     int64  
 7   original_language  23 non-null     int64  
 8   action             23 non-null     int64  
 9   adventure          23 non-null     int64  
 10  comedy             23 non-null     int64  
 11  crime              23 non-null     int64  
 12  drama              23 non-null     int64  
 13  family             23 non-null     int64  
 14  fantasy            23 non-null     int64  
 15  history            23 non-null     int64  
 16  horror             23 non-null 

In [16]:
# Ensure the column order matches the expected order
expected_columns = ['release_year', 'runtime', 'budget', 'original_language', 'first_company', 'first_producer', 'first_actor', 'imdb_votes', 'action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction', 'tv_movie', 'thriller', 'war', 'western']
train_features=train_features[expected_columns]

train_features

,release_year,runtime,budget,original_language,first_company,first_producer,first_actor,imdb_votes,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,history,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western
57,2025,0.0,75000000.0,0,5,4,18,400,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
60,2026,0.0,85000000.0,0,11,7,10,400,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
100,2024,0.0,290000000.0,4,8,3,22,400,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
108,2025,0.0,68000000.0,0,14,18,4,400,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
198,2025,196.0,22000000.0,0,20,0,7,400,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
203,2024,0.0,190000000.0,4,10,3,6,400,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0
252,2024,92.0,12500000.0,0,3,15,17,400,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
257,2025,0.0,18218260.0,3,19,16,12,400,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
308,2025,0.0,155000000.0,0,13,5,20,400,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0
316,2025,0.0,70000000.0,0,1,8,21,400,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
predicted = retrieved_xgboost_model.predict(train_features)

In [18]:
train_features["predicted_rating"]=predicted

train_features

2025-01-08 13:40:46,298 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,release_year,runtime,budget,original_language,first_company,first_producer,first_actor,imdb_votes,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,history,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western,predicted_rating
57,2025,0.0,75000000.0,0,5,4,18,400,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,4.438272
60,2026,0.0,85000000.0,0,11,7,10,400,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.085915
100,2024,0.0,290000000.0,4,8,3,22,400,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.385547
108,2025,0.0,68000000.0,0,14,18,4,400,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3.843566
198,2025,196.0,22000000.0,0,20,0,7,400,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,7.389141
203,2024,0.0,190000000.0,4,10,3,6,400,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,4.432975
252,2024,92.0,12500000.0,0,3,15,17,400,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3.828052
257,2025,0.0,18218260.0,3,19,16,12,400,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.136247
308,2025,0.0,155000000.0,0,13,5,20,400,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,4.804152
316,2025,0.0,70000000.0,0,1,8,21,400,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.279350


In [19]:
df_rated = pd.concat([df_b, train_features], axis=1)

df_rated

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year,release_year,runtime,budget,original_language,first_company,first_producer,first_actor,imdb_votes,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,history,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western,predicted_rating
57,1100988,28 Years Later,0.0,0.0,Post Production,2025-06-16 00:00:00+00:00,0.0,0.0,75000000.0,tt10548174,en,28 Years Later,It's been almost three decades since the rage ...,79.863,In 28 days it began. In 28 weeks it spread. In...,"Horror, Science Fiction","DNA Films, Decibel Films, Columbia Pictures, BFI","United Kingdom, United States of America",English,"Ralph Fiennes, Joe Blakemore, Nathan Hall, Edv...",Danny Boyle,Anthony Dod Mantle,"Danny Boyle, Alex Garland","Danny Boyle, Peter Rice, Cillian Murphy, Berna...",None,NaN,400,/hVKHzr4GwSw0FepqhqQ0DDiYHNY.jpg,2025,2025.0,0.0,75000000.0,0.0,5.0,4.0,18.0,400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.438272
60,1101383,Flowervale Street,0.0,0.0,Post Production,2026-03-12 00:00:00+00:00,0.0,0.0,85000000.0,tt27165187,en,Flowervale Street,A family in the '80s start to notice bizarre h...,4.194,None,"Adventure, Thriller","Jackson Pictures, Bad Robot, Warner Bros. Pict...",United States of America,English,"Jordan Alexa Davis, Anne Hathaway, Christian C...",David Robert Mitchell,Mike Gioulakis,David Robert Mitchell,"David Robert Mitchell, Chris Bender, J.J. Abra...",None,NaN,400,None,2026,2026.0,0.0,85000000.0,0.0,11.0,7.0,10.0,400.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.085915
100,1294480,Генг Бенг 3,0.0,0.0,Post Production,2024-05-27 00:00:00+00:00,0.0,0.0,290000000.0,None,ru,Генг Бенг 3,None,0.000,None,"Action, Comedy, Adventure",GangBang Studios Co.,None,None,"Гомункул, Cillian Murphy, Ryan Gosling, Ryan R...",Хлазу,None,Хлазу,"Christopher Nolan, Хлазу",None,NaN,400,None,2024,2024.0,0.0,290000000.0,4.0,8.0,3.0,22.0,400.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.385547
108,931285,Mortal Kombat 2,0.0,0.0,Post Production,2025-10-23 00:00:00+00:00,0.0,0.0,68000000.0,tt17490712,en,Mortal Kombat 2,A sequel to the 2021 film 'Mortal Kombat'.,16.116,None,"Action, Fantasy, Adventure","New Line Cinema, Atomic Monster, Broken Road P...",United States of America,English,"Desmond Chiam, Tati Gabrielle, Josh Lawson, Jo...",Simon McQuoid,Stephen F. Windon,Jeremy Slater,"Michael Clear, Simon McQuoid, Toby Emmerich, J...",Benjamin Wallfisch,NaN,400,/yN1bQ8XLh7Q2bEu3YBtp68vUeXl.jpg,2025,2025.0,0.0,68000000.0,0.0,14.0,18.0,4.0,400.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.843566
198,1277016,The Queen of Heaven,0.0,0.0,Post Production,2025-04-04 00:00:00+00:00,0.0,196.0,22000000.0,tt25049058,en,Η βασίλισσα των ουρανών,The life of the Virgin Mary from the Crucifixi...,2.189,None,Drama,"Y.S. Pictures, Crisis Cinema, Enorama, BO Film...",Greece,English,"George Angelopoulos, Maria Aliferi, Yorgo Cons...",Yiannis Stravolaimos,None,Yiannis Stravolaimos,"Ali Bakkioui, Yiannis Stravolaimos",None,NaN,400,/zY2vIC5q9waeSKDrYjAQnP6CyTz.jpg,2025,2025.0,196.0,22000000.0,0.0,20.0,0.0,7.0,400.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.389141
203,1292465,Совет Андреев,0.0,0.0,Post Production,2024-08-09 00:00:00+00:00,0.0,0.0,190000000.0,None,ru,Совет Андреев,None,1.400,None,"Thriller, Comedy, Drama, Science Fiction, Fantasy","JPD4, GangBang Studios Co.",Russia,None,"Ezra Miller, Ryan Reynolds, Cillian Murphy, To...","JPD4, Дариус",None,Дариус,"Christopher Nolan, Дариус",None,NaN,400,None,2024,2024.0,0.0,190000000.0,4.0,10.0,3.0,6.0,400.0,0.0

In [20]:
df_rated.to_csv("movies.csv", index=False)